# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read Data from the CSV File
weather_data_df = pd.read_csv('weather_data.csv')
weather_data_df.head(20)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tsimmerbude,94,RU,1584475467,80,54.67,20.13,41.00,6.71
1,Butaritari,1,KI,1584475470,76,3.07,172.79,82.85,14.23
2,Puerto Ayora,20,EC,1584475471,66,-0.74,-90.35,84.20,11.41
3,Cabo San Lucas,75,MX,1584475472,53,22.89,-109.91,82.40,3.36
4,Namatanai,100,PG,1584475474,78,-3.67,152.43,81.95,8.68
5,Punta Arenas,75,CL,1584475200,50,-53.15,-70.92,57.20,29.97
6,Doha,0,QA,1584475456,78,25.29,51.53,73.99,10.29
7,Novoagansk,100,RU,1584475479,96,61.94,76.66,21.29,1.83
8,Severo-Kuril'sk,100,RU,1584475481,91,50.68,156.12,17.15,11.23
9,New Norfolk,75,AU,1584475483,49,-42.78,147.06,68.00,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Location
locations = weather_data_df[["Lat", "Lng"]]

humidity = weather_data_df["Humidity"]

In [4]:
# Heatmap Plot
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=2)

# Add Layer
fig.add_layer(heat_layer)

# Save the Fig Plot
plt.savefig("vacationheatmap.png")

# Display
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow Down by weather conditions
first = weather_data_df.loc[weather_data_df["Max Temp"] < 80, :]
second = first.loc[weather_data_df["Max Temp"] > 70, :]
second.head()

# Narrow by it wind speed
third = second.loc[second["Wind Speed"] < 10, :]
third

# Narrow by Cloudiness
fourth = third.loc[third["Cloudiness"]==0, :]
fourth.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
46,Castro,0,BR,1584475458,80,-24.79,-50.01,72.05,5.39
155,Rock Sound,0,BS,1584475782,79,24.90,-76.20,73.67,8.93
190,Sarishābāri,0,BD,1584475856,42,24.73,89.83,70.79,1.59
191,Talhar,0,PK,1584475859,26,24.88,68.82,72.77,6.31
207,Puri,0,IN,1584475897,88,19.80,85.85,78.80,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Tsimmerbude,RU,54.67,20.13,
1,Butaritari,KI,3.07,172.79,
2,Puerto Ayora,EC,-0.74,-90.35,
3,Cabo San Lucas,MX,22.89,-109.91,
4,Namatanai,PG,-3.67,152.43,
...,...,...,...,...,...
564,Urubichá,BO,-15.38,-62.95,
565,Iberia,US,29.95,-91.75,
566,Yabēlo,ET,4.88,38.08,
567,Vysokogornyy,RU,50.12,139.15,


In [ ]:
# Set parameters to search per above
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df.info(10)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Display Map
fig.add_layer(heat_layer)

fig.add_layer(hotel_layer)

# Save the Heat Map
plt.savefig("HotelMap.png")

fig